In [3]:
import theano as th
import theano.tensor as T
import numpy as np
import matplotlib.pyplot as plt

In [32]:
class LogisticRegressionEstimator:
    def __init__(self, n_input, n_output):
        self.input_shape = n_input
        self.output_shape = n_output
        
        self.weight = th.shared(value=np.asarray(
                                np.random.randn(n_input, n_output),
                                dtype=th.config.floatX), name='weight', 
                               borrow=True)
        self.bias = th.shared(value=np.zeros((n_output, ), dtype=th.config.floatX),
                             name='bias', borrow=True)
        self.params = [self.weight, self.bias]
        
        self.input = T.matrix('x')
        self.target = T.ivector('t')
        
        self.prob = T.nnet.softmax(T.dot(self.input, self.weight) + self.bias)
        self.pred = T.argmax(self.prob, axis=1)
        
        self.loss = -T.mean(T.log(self.prob[T.arange(self.target.shape[0]), self.target]))
        self.gparams = T.grad(cost=self.loss, wrt=self.params)
        self.accuracy = T.mean(T.eq(self.pred, self.target))
        self.predictor = th.function(inputs=[self.input],
                                    outputs=[self.pred])
              
    def fit(self, x, y, epoch, learning_rate, show_intermediate=False):
        self.updates = []
        for param, gparam in zip(self.params, self.gparams):
            param_new = param - learning_rate * gparam
            self.updates.append((param, param_new))
            
        self.estimator = th.function(inputs=[self.input, self.target],
                                    outputs=[self.loss, self.accuracy],
                                    updates=self.updates)
        train_hist = {
            'train_acc': [],
            'train_loss': []
        }
        
        import timeit
        start_time = timeit.default_timer()
        for cur_epoch in range(epoch):
            loss, acc = self.estimator(x, y)
            train_hist['train_acc'].append(acc)
            train_hist['train_loss'].append(loss)
            
            if (cur_epoch + 1) % (epoch / 50) and show_intermediate:
                print('Loss: {}, Accuracy: {}'.format(loss, acc))
                
        end_time = timeit.default_timer()
        print('Loss: {}, Accuracy: {}, time elapsed {}'.format(train_hist['train_acc'][-1],
                                                              train_hist['train_loss'][-1],
                                                              end_time - start_time))
        
    def predict(self, x):
        return self.predictor(x)
    
    def evaluate(self, x, y):
        return self.estimator(x, y)

In [88]:
from keras import datasets
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

In [89]:
x_train = x_train.reshape(x_train.shape[0], 
                         np.prod(x_train.shape[1:]))

x_test = x_test.reshape(x_test.shape[0], 
                         np.prod(x_test.shape[1:]))

In [90]:
logit_classifier = LogisticRegressionEstimator(np.prod(x_train.shape[1:]), 10)

In [91]:
logit_classifier.fit(x_train, y_train, learning_rate=0.01, epoch=10000)

Loss: 0.9290666666666667, Accuracy: 37.7731819152832, time elapsed 1720.5492778560001


In [92]:
logit_classifier.evaluate(x_test, y_test)

[array(55.130314, dtype=float32), array(0.9163)]

In [93]:
logit_classifier.predict(x_test[:10])

[array([7, 2, 1, 0, 4, 1, 4, 9, 6, 9])]

In [94]:
y_test[:10]

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9], dtype=uint8)

# MLP

In [2]:
import theano_utils

# LeNet

In [5]:
import theano_utils
from keras import datasets

Using TensorFlow backend.


In [6]:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

In [4]:
x = T.matrix('x')
t = T.ivector('y')

In [27]:
batch_size = 500
x_input = x.reshape((batch_size, 1) + x_train[0].shape)

In [28]:
conv1 = theano_utils.conv_layer(x_input, input_shape=(batch_size, 1) + x_train[0].shape,
                               filter_shape=(5, 5), n_filters=32)
conv1_out = conv1()
conv1_pooled = T.signal.pool.pool_2d(input=conv1_out, ws=(2, 2), ignore_border=True)

In [29]:
conv2 = theano_utils.conv_layer(conv1_pooled, input_shape=(batch_size, 1) + (12, 12),
                               filter_shape=(5, 5), n_filters=64)
conv2_out = conv2()
conv2_pooled = T.signal.pool.pool_2d(input=conv2_out, ws=(2, 2), ignore_border=True)

In [30]:
flattened = conv2_pooled.flatten(2)

In [31]:
fc1 = theano_utils.fc_layer(flattened, 4*4*64, 500)
fc1_out = fc1()